# Discrete-Time Systems

## 1. Oefeningen wk 1

In [1]:
import numpy as np
from numpy.linalg import det, inv, eig
print("Question 2:", det(np.array([[5., -2.], [-2., 8.]])))
print("Question 3:", inv(np.array([[8., 2.], [2., 5.]])))
print("Question 3 (36 times):", 36 * inv(np.array([[8., 2.], [2., 5.]])))
print("Question 4a:", eig(np.array([[0., 1.], [1., 0.]]))[0])
print("Question 4b:", eig(np.array([[0., 1.], [-1., 0.]]))[0])
print("Question 4c:", eig(np.array([[0., 1.], [-25., -6.]]))[0])

Question 2: 36.0
Question 3: [[ 0.13888889 -0.05555556]
 [-0.05555556  0.22222222]]
Question 3 (36 times): [[ 5. -2.]
 [-2.  8.]]
Question 4a: [ 1. -1.]
Question 4b: [0.+1.j 0.-1.j]
Question 4c: [-3.+4.j -3.-4.j]


### Begrippen sampling (bemonsteren), zie verder simulatie vorige week
Sampling Continuous-Time Signals (§2.2)
Bemonsteren op discrete tijdstippen $t_k$ voor $k=\cdots,-2,-1,0,1,2,\cdots$ ofwel $k\in\mathbb{Z}$.

Meestal: equidistant sampling
$$ t_k ~ = ~ k\cdot h$$
Sampling period, sample tijd: $h$

Sampling frequency: 
\begin{align}
f_s &=1/h & \mbox{(in Hz)}\\
\omega_s &=2\pi/h & \mbox{(in rad/s)}
\end{align}
Nyquist frequency is helft van de bemonster frequentie:
\begin{align}
f_N &=1/(2h) & \mbox{(in Hz)}\\
\omega_N &=\pi/h & \mbox{(in rad/s)}
\end{align}


In [2]:
# Controle voorbeelden uit college van matrix exponent functie
from scipy.linalg import expm      # matrix exp functie
from numpy import exp as exp_numpy # ter vergelijking, exp in numpy doet element-wise exponentiation(!)
print(expm(np.zeros((3,3))))
#print(expm(np.eye(3)))
#print(expm(np.array([[0.,1.],[0.,0.]])))
#print(expm(np.array([[2.,0.],[0.,-1.]])))
#print(expm(np.array([[0.,1.],[1.,0.]])))

#print("With numpy (no matrix exponential function!):")
#print(exp_numpy(np.zeros((3,3))))
#print(exp_numpy(np.eye(3)))
#print(exp_numpy(np.array([[0.,1.],[0.,0.]])))
#print(exp_numpy(np.array([[2.,0.],[0.,-1.]])))
#print(exp_numpy(np.array([[0.,1.],[1.,0.]])))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## `mbrtc.py`

In [3]:
# Author:  Rufus Fraanje
# Email:   p.r.fraanje@hhs.nl
# Licence: GNU General Public License (GNU GPLv3)
# Creation date: 2019-03-06
# Last modified: 2019-04-18

import numpy as np
from scipy.linalg import expm, logm
from scipy.signal import tf2ss, ss2tf

# Symbols (variations may be made for clarity, depending on context,
# e.g. ud to stress that the input is in discrete-time):
#
# tc          - continuous time instants
# td          - discrete time instants (1-D array)
# h           - sampling time
# u           - input signal (1-D array for single channel signals, for
#                             multichannels 2-D array where each column
#                             is at one sample instant)
# y           - output signal (idem)
# x0          - initial state
# x           - state sequence
# NS          - number of samples
# ni          - number of inputs
# no          - number of outputs
# ns          - number of states
# Ac,Bc,Cc,Dc - continuous time state-space models
#    Ac       - state-transition matrix (ns x nx) (A in Computer-Controlled Systems)
#    Bc       - input matrix (ns x ni) (B in Computer-Controlled Systems)
#    Cc       - output matrix (no x ns) (C in Computer-Controlled Systems)
#    Dc       - direct feedthrough matrix (no x ni) (D in Computer-Controlled Systems)
# Ad,Bd,Cd,Dd - discrete time state-space models
#    Ad       - state-transition matrix (ns x nx) (Φ (Phi) in Computer-Controlled Systems)
#    Bd       - input matrix (ns x ni) (Γ (Gamma) in Computer-Controlled Systems)
#    Cd       - output matrix (no x ns) (C in Computer-Controlled Systems)
#    Dd       - direct feedthrough matrix (no x ni) (D in Computer-Controlled Systems)


def spike(NS=100,at_sample=1):
    signal = np.zeros((NS,1))
    signal[at_sample,0] = 1.
    return signal

def random_impulses(av_samples_per_spike=10,NS=100,nchan=1):
    signal = np.zeros((NS,nchan))
    for i in range(nchan):
        signal[:,i] = np.floor(np.random.randint(0,av_samples_per_spike+1,NS)/av_samples_per_spike)
    return signal

def c2d_zoh_AB(Ac,Bc,h):
    AB = np.hstack((Ac,Bc))
    ns,ni = Bc.shape
    lower_lines = np.zeros((ni,ns+ni))
    ABext = np.vstack((AB,lower_lines))*h
    ABext_zoh = expm(ABext)
    Ad = ABext_zoh[0:ns,0:ns]
    Bd = ABext_zoh[0:ns,ns:ns+ni]
    return Ad,Bd

def d2c_zoh_AB(Ad,Bd,h):
    AB_zoh = np.hstack((Ad,Bd))
    ns,ni = Bd.shape
    lower_lines = np.hstack((np.zeros((ni,ns)),np.eye(ni)))
    ABext_zoh = np.vstack((AB_zoh,lower_lines))
    ABext = 1/h * logm(ABext_zoh)
    Ac = ABext[0:ns,0:ns]
    Bc = ABext[0:ns,ns:ns+ni]
    return Ac,Bc


def c2d_zoh(Ac,Bc,Cc,Dc,h):
    Ad,Bd = c2d_zoh_AB(Ac,Bc,h)
    Cd = Cc.copy()
    Dd = Dc.copy()
    return Ad,Bd,Cd,Dd

def d2c_zoh(Ad,Bd,Cd,Dd,h):
    Ac,Bc = d2c_zoh_AB(Ad,Bd,h)
    Cc = Cd.copy()
    Dc = Dd.copy()
    return Ac,Bc,Cc,Dc

def similarity_trans(A,B,C,T):
    """ Similarity transformation, gives 
        state-space representation matrices At,Bt,Ct
        of system with transformed state: 
          x_T = T x """
    Ti = np.linalg.inv(T)
    At = T @ A @ Ti
    Bt = T @ B
    Ct = C @ Ti
    return At,Bt,Ct

def sim_state(A,B,u,x0=None):
    # Discrete-time forced state iteration:
    ns,ni=B.shape
    reshape_state = False
    if len(u.shape)==1: # u is a 1-dim signal
        u = np.reshape(u,(1,u.shape[0]))
        if ns==1: # make state a 1-dim array as input was
            reshape_state = True
    else:
        if u.shape[0]>u.shape[1]:
            Warning('u is not wide, u should have its samples on each column!')
    N=u.shape[1]
    if x0==None: x0 = np.zeros((ns))
    x = np.zeros((ns,N))
    x[:,0] = x0
    for i in range(N-1):
        x[:,i+1] = A @ x[:,i] + B @ u[:,i]
    if reshape_state:
        x = np.reshape(x,(N))
    return x


def sim(A,B,C,D,u,x0=None,return_X=False):
    # Discrete-time forced state-space model iteration
    no,ni=D.shape
    ns=A.shape[0]
    reshape_output = False
    reshape_state = False
    if len(u.shape)==1: # u is a 1-dim signal
        u = np.reshape(u,(1,u.shape[0]))
        if no==1: # make output a 1-dim array as input was
            reshape_output = True
        if ns==1: # make state a 1-dim array as input was
            reshape_state = True
    else:
        if u.shape[0]>u.shape[1]:
            Warning('u is not wide, u should have its samples on each column!')
    N=u.shape[1]
    x = sim_state(A,B,u,x0)
    if reshape_state==1:
        x = np.reshape(x,(ns,N))
    y = C @ x + D @ u
    if reshape_output:
        y = np.reshape(y,(N))
    if return_X:
        if reshape_state:
            x = np.reshape(x,(N))
        return y,x
    else:
        return y



def acker(A,B,P):
    raise NotImplementedError
    return None

def c2d_pole(lambda_i,h):
    return np.exp(lambda_i*h)

#TODO
def canon(A,method):
    # see section 2 in Feng Ding, Transformations between some special matrices,
    # Computers and Mathematics with Applications 59(2010), 2676-2694.
    n = np.shape(A)[0]
    if method=="ctrl":
        T2inv = np.zeros((n,n))
        T2inv[n-1:n,:] = np.hstack((np.array([[1.]]),np.zeros((1,n-1))))
        for i in range(n-1):
            T2inv[n-i-2:n-i-1,:] = T2inv[n-i-1:n-i,:] @ A

    return T2inv



def ctrb(A,B):
    """ Returns controllability matrix:
        Wc = [B AB ... A^(n-1) B]
        """
    ns,ni = np.shape(B)
    Wc = np.zeros((ns,ns*ni));
    Wc[:,0:ni] = B
    for i in range(n-1):
        Wc[:,(1+i)*ni:(2+i)*ni] =  A @ Wc[:,i*ni:(1+i)*ni]
    return Wc

def obsv(A,C):
    """ Returns observability matrix:
        Wo = [C
              CA
              ...
              CA^(n-1)]
        """
    no,ns = np.shape(C)
    Wo = np.zeros((ns*no,ns));
    Wo[0:no,:] = C
    for i in range(n-1):
        Wo[(1+i)*no:(2+i)*no,:] =  Wo[i*ni:(1+i)*ni,:] @ A
    return Wo

def c2d_characteristic_equation(ac,h):
    n = np.max(np.shape(ac))-1
    ac_adj = ac[1:].reshape((1,n))/ac[0]
    Ac = np.diag(np.ones((n-1)),-1)
    Ac[0:1,:] = -ac_adj
    Ad = expm(Ac*h)
    ad = np.poly(Ad)
    # T2inv = canon(Ad,"ctrl")
    # Ad_companion = T2inv @ Ad @ np.linalg.inv(T2inv)
    # ad = np.hstack((np.array([1.]),-Ad_companion[0,:]))
    return ad



def c2d_zoh_intersample(Ac,Bc,Cc,Dc,h,number_intersamples):
    ns = Ac.shape[0]
    no,ni = Dc.shape
    Ad,Bd = c2d_zoh_AB(Ac,Bc,h)
    Cd = np.zeros((number_intersamples*no,ns))
    Dd = np.zeros((number_intersamples*no,ni))
    for i in range(number_intersamples):
        if i==0:
            Cd[0:no,:] = Cc
            Dd[0:no,:] = Dc
        else:
            Ad_interstep,Bd_interstep = c2d_zoh_AB(Ac,Bc,i*h/number_intersamples)
            Cd[i*no:(i+1)*no,:] = Cc @ Ad_interstep
            Dd[i*no:(i+1)*no,:] = Cc @ Bd_interstep + Dc
    return Ad,Bd,Cd,Dd


def sim_intersample(Ac,Bc,Dc,Dd,h,number_is,ud,td,x0=None):
    """ Intersample simulation of a continuous state space model
        (Ac,Bc,Cc,Dc) discretised with zero-order-hold with sampling
        time h. The parameter number_is determines the number of (equidistant) 
        intersamples. ud is the zero-order hold input, x0 is the initial
        state (None for zero initial state)."""
    Ad_is,Bd_is,Cd_is,Dd_is = c2d_zoh_intersample(Ac,Bc,Cc,Dc,h,number_is)
    NS = len(td)
    yd_is = sim(Ad_is,Bd_is,Cd_is,Dd_is,ud)
    td_is = np.repeat(td,number_is) + np.tile(h/number_is * np.array(range(number_is)), NS)
    yd_is = np.reshape(yd_is,(NS*number_is),order='F')
    return td_is,yd_is


def ss_series(A1,B1,C1,D1,A2,B2,C2,D2):
    """Series interconnection of two state-space models:
        --> (A1,B1,C1,D1) --> (A2,B2,C2,D2) -->
        state of series connection is: [ x1 ]
                                       [ x2 ]
        """
    n1 = np.shape(A1)[0]
    n2 = np.shape(A2)[0]
    A = np.vstack( (np.hstack( (A1,np.zeros(n1,n2)) ), np.hstack( (B2@C1,A2) )  ) )
    B = np.vstack( (B1,B2@D1) )
    C = np.hstack( (D2@C1, C2) )
    D = D2@D1
    return (A,B,C,D)

def ss_parallel(A1,B1,C1,D1,A2,B2,C2,D2,sign=None):
    """Parallel interconnection of two state-space models:
                            +
        ---> (A1,B1,C1,D1) -->sum ---->
         |                     ^sign
         |-> (A2,B2,C2,D2) ----|

         sign is a list of signs, if None, than all signs are positive 

        state of parallel connection is: [ x1 ]
                                         [ x2 ]
        """
    no = np.shape(D2)[1]
    if not(sign): sign = np.ones((1,no)) 
    signMat = np.diag(sign)
    n1 = np.shape(A1)[0]
    n2 = np.shape(A2)[0]
    A = np.vstack( (np.hstack( (A1,np.zeros(n1,n2)) ), np.hstack( (np.zeros(n2,n1),A2) )  ) )
    B = np.vstack( (B1,B2) )
    C = np.hstack( (C1, signMat@C2) )
    D = D1+signMat@D2
    return (A,B,C,D)


def ss_feedback(A1,B1,C1,D1,A2,B2,C2,D2,sign=None):
    """Feedback interconnection of two state-space models:

        ---> sum --> (A1,B1,C1,D1) ---->
         sign ^                     |
              |-- (A2,B2,C2,D2) <---|

        sign is a list of signs, if None, than all signs are negative 

        state of feedback connection is: [ x1 ]
                                         [ x2 ]
        """
    no,ni = np.shape(D1)
    if not(sign): sign = -np.ones((1,ni))
    signMat = np.diag(sign)
    DDinv = np.linalg.pinv(np.eye(no)+(signMat@D1)@D2)
    Acl11 = A1+(signMat@B1)@D2@DDinv@C1
    Acl12 = (signMat@B1)@C2+(signMat@B1)@D2@DDinv*(signMat@D1)@C2
    Bcl1 = B1 +(signMat@B1)@D2@DDinv@D1
    Acl21 = B2@DDinv@C1
    Acl22 = A2+B2@DDinv@(signMat@D1)@C2
    Bcl2 = B2@DDinv@D1
    Ccl1 = DDinv@C1
    Ccl2 = DDinv@(signMat@D1)@C2
    Acl = np.vstack(  (np.hstack((A11,A12)),np.hstack((A21,A22))) )
    Bcl = np.vstack( (Bcl1,Bcl2) )
    Ccl = np.hstack( (Ccl1,Ccl2) )
    Dcl = DDinv@D1
    return (Acl,Bcl,Ccl,Dcl)
